# Dumbed Down Model Used To Get Flask Up And Running 

In [1]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import string, re
import urllib
import seaborn as sbn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from nltk.corpus import stopwords

In [2]:
data = pd.read_csv('../../Data/2019/listings122019long.csv')

In [3]:
data1 = data.drop(columns=['listing_url','scrape_id','last_scraped','summary','space','description','experiences_offered',
                           'neighborhood_overview','notes','transit','access','interaction','house_rules',
                           'thumbnail_url','medium_url','picture_url','xl_picture_url','host_id','host_url',
                           'host_name','host_since', 'host_location','host_about','host_response_time','host_response_rate',
                           'host_acceptance_rate', 'host_thumbnail_url','host_picture_url', 'host_neighbourhood',
                           'host_listings_count','host_total_listings_count', 'host_verifications','host_has_profile_pic',
                           'host_identity_verified','street', 'city','state','zipcode','market','country_code',
                           'country', 'is_location_exact','bed_type','amenities','square_feet','weekly_price',
                           'monthly_price','security_deposit','guests_included','extra_people','maximum_nights',
                           'minimum_minimum_nights','maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                           'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
                           'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped','first_review',
                           'last_review','review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                           'review_scores_communication', 'review_scores_location','requires_license', 'license',
                           'jurisdiction_names', 'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
                           'require_guest_profile_picture', 'require_guest_phone_verification', 'calculated_host_listings_count',
                           'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                           'calculated_host_listings_count_shared_rooms', 'neighbourhood','smart_location','id','property_type',
                           'neighbourhood_cleansed','name','room_type','minimum_nights','number_of_reviews_ltm',
                           'review_scores_rating','review_scores_value','reviews_per_month'
                          ])

In [4]:
data1['price'] = data1['price'].str.extract('(\d+)', expand=False).astype('float')
data1['cleaning_fee'] = data1['cleaning_fee'].str.extract('(\d+)', expand=False).astype('float')
data1['host_is_superhost'] = (data1['host_is_superhost'] == 't').astype('int')
data1['bathrooms'] = data1['bathrooms'].fillna(data1['bathrooms'].mean())
data1['bedrooms'] = data1['bedrooms'].fillna(data1['bedrooms'].mean())
data1['beds'] = data1['beds'].fillna(data1['beds'].mean())
data1['cleaning_fee'] = data1['cleaning_fee'].fillna(data1['cleaning_fee'].mean())
data1['longitude'] = data1['longitude'].round(decimals=5)
data1['latitude'] = data1['latitude'].round(decimals=5)
data1 = data1[data1.number_of_reviews>0]

In [5]:
ohe_ng = OneHotEncoder(sparse=False)
neigh_group = ohe_ng.fit_transform(data1[['neighbourhood_group_cleansed']])
neigh_group_cat = ohe_ng.categories_

In [6]:
def rename(name_of_columns,pre_addition):
    new_list = []
    for x in name_of_columns:
        for x in x:
            new_list.append(pre_addition+ '' + x)
    return new_list

In [7]:
new_neigh_group_cat = rename(neigh_group_cat,'neighbourhood_group: ')
neigh_group_df = pd.DataFrame(data=neigh_group,columns=new_neigh_group_cat)

In [8]:
clean_data = pd.concat([data1,neigh_group_df],axis=1)
clean_data = clean_data.drop(columns=['neighbourhood_group_cleansed'])

In [9]:
abcdefg = []
for n in neigh_group:
    for n in n:
            abcdefg.append(n)
type(abcdefg[0])

numpy.float64

In [46]:
clean_data

,host_is_superhost,accommodates,bathrooms,bedrooms,beds,cleaning_fee,number_of_reviews,neighbourhood_group: Ballard,neighbourhood_group: Beacon Hill,neighbourhood_group: Capitol Hill,...,neighbourhood_group: Northgate,neighbourhood_group: Other neighborhoods,neighbourhood_group: Queen Anne,neighbourhood_group: Rainier Valley,neighbourhood_group: Seward Park,neighbourhood_group: University District,neighbourhood_group: West Seattle,longitude,latitude,price
0,1.0,9.0,2.5,4.0,4.0,250.000000,31.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.29082,47.61082,296.0
1,0.0,2.0,1.0,0.0,1.0,45.000000,462.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.35989,47.52398,48.0
2,0.0,2.0,1.0,1.0,1.0,40.000000,150.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-122.33761,47.65411,90.0
3,1.0,2.0,3.0,1.0,1.0,0.000000,147.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-122.32014,47.55062,62.0
4,1.0,2.0,1.0,1.0,1.0,45.000000,457.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.32936,47.61265,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6852,0.0,4.0,1.0,0.0,2.0,50.000000,4.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.31148,47.61395,107.0
6853,1.0,4.0,2.0,2.0,2.0,135.000000,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.36061,47.56733,150.0
6854,0.0,4.0,1.0,2.0,2.0,83.811275,4.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.35983,47.51957,90.0
6855,0.0,4.0,1.0,1.0,2.0,44.000000,40.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.38591,47.57403,109.0


In [11]:
clean_data = clean_data[clean_data.price>20]
clean_data = clean_data[clean_data.price<800]
clean_data = clean_data.dropna()

In [12]:
cols = ['host_is_superhost', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'cleaning_fee', 'number_of_reviews',
       'neighbourhood_group: Ballard', 'neighbourhood_group: Beacon Hill',
       'neighbourhood_group: Capitol Hill', 'neighbourhood_group: Cascade',
       'neighbourhood_group: Central Area', 'neighbourhood_group: Delridge',
       'neighbourhood_group: Downtown', 'neighbourhood_group: Interbay',
       'neighbourhood_group: Lake City', 'neighbourhood_group: Magnolia',
       'neighbourhood_group: Northgate',
       'neighbourhood_group: Other neighborhoods',
       'neighbourhood_group: Queen Anne',
       'neighbourhood_group: Rainier Valley',
       'neighbourhood_group: Seward Park',
       'neighbourhood_group: University District',
       'neighbourhood_group: West Seattle','longitude','latitude','price']

In [13]:
clean_data = clean_data[cols]

In [15]:
X = clean_data.drop(columns=['price'])
y = clean_data['price']

In [44]:
rfr = RandomForestRegressor(n_estimators=755,min_samples_split=5,min_samples_leaf=3,random_state=11)
rfr.fit(X,y)
rfr.score(X,y)

0.8809760616961901

In [45]:
import pickle
pickle.dump(rfr, open('rftest2.sav', 'wb'))

In [ ]:
X.columns

In [ ]:
rfr.feature_importances_

In [ ]:
X